In [1]:

from model import RNNModel

In [2]:
import numpy as np
import torch
from utils import generate_data, check_data,word_ids_to_sentence
import torchtext
from torchtext import data
import torch.nn as nn
import torch.nn.functional as F
from train_eval import train,test

from torch.autograd import Variable as V

from model import RNNModel

class Config(object):
    def __init__(self,data_ori):
        self.model_name="lm_model"
        self.data_ori=data_ori+"E:/data/word_nlp/cnews_data/"
        self.train_path="train_0.csv"
        self.valid_path="train_0.csv"
        self.test_path="test_100.csv"
        self.sen_max_length=150
        #self.embedding_path = "need_bertembedding"
        self.embedding_path = "bert_embedding"
        self.embedding_dim=768
        self.vocab_maxsize=4000
        self.vocab_minfreq=10
        self.save_path="lm_ckpt"

        self.batch_size = 64
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.hidden_size=200
        self.nlayers=1
        self.dropout=0.5
        self.epoch=2

        self.train_len=0
        self.test_len = 0
        self.valid_len = 0
        
        self.mode="test"
        
#data_path="E:/study_series/2020_3/re_write_classify/data/"
#data_path="/mnt/data3/wuchunsheng/code/nlper/NLP_task/text_classification/my_classification_cnews/2020_3_30/text_classify/data/"
data_path=""
config=Config(data_path)
train_iter, valid_iter, test_iter, TEXT=generate_data(config)

example len::   1
train_examples[0] len:  4470802
valid_examples[0] len:  4470802
词汇量：  2903


In [3]:
model = RNNModel(config, TEXT).to(config.device)

C:\Love\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [4]:
model.load_state_dict(torch.load(config.save_path))

<All keys matched successfully>

In [5]:
from train_eval import test

In [6]:
#test(config, model, TEXT,test_iter)

In [7]:
b=next(iter(test_iter))
print(b)


[torchtext.data.batch.Batch of size 64]
	[.text]:[torch.cuda.LongTensor of size 50x64 (GPU 0)]
	[.target]:[torch.cuda.LongTensor of size 50x64 (GPU 0)]


In [27]:
b.text[:,0].shape

torch.Size([50])

In [28]:
b.target[0]
print("".join([TEXT.vocab.itos[x] for x in b.text[:,0]]))
print("".join([TEXT.vocab.itos[x] for x in b.target[:,0]]))

context,label<unk>id<eos>"鲍勃库西奖归谁属？ NCAA最强控卫是坎巴还是弗神新浪体育讯如今
ontext,label<unk>id<eos>"鲍勃库西奖归谁属？ NCAA最强控卫是坎巴还是弗神新浪体育讯如今，


In [32]:
b.text.shape
b.text[0:3,0]

tensor([1174,  699,  951], device='cuda:0')

In [10]:
b.text[:,0].view(-1,1).shape

torch.Size([50, 1])

In [12]:
arrs=model(b.text[:,0].view(-1,1))

In [13]:
arrs.shape

torch.Size([50, 1, 15783])

In [16]:
preds = word_ids_to_sentence(np.argmax(arrs.detach().cpu(), axis=2), TEXT.vocab)

<class 'torch.Tensor'>


In [33]:
"".join(preds)

'卡ms,r体o0onP o ny",里-斯斯     他BAAA佳队卫新新比斯是一朗的浪体育讯北今的'

In [35]:
arrs=model(b.text[0:3,0].view(-1,1))
preds = word_ids_to_sentence(np.argmax(arrs.detach().cpu(), axis=2), TEXT.vocab)
"".join(preds)

<class 'torch.Tensor'>


'om)'

In [52]:
sentence="体育的人"
def test_sentence(model , TEXT, sentence):
    inputs=torch.Tensor([TEXT.vocab[one] for one in sentence]).long().to(config.device)
    inputs=inputs.view(-1,1)
    #print(inputs.shape)
    arrs=model(inputs)
    preds = word_ids_to_sentence(np.argmax(arrs.detach().cpu(), axis=2), TEXT.vocab)
    
    return preds
test_sentence(model, TEXT, sentence)

<class 'torch.Tensor'>


['育', '馆', '伤', '，']

In [24]:
import codecs
path="E:/data/word_nlp/cnews_data/bert_embedding"

lines = codecs.open(path, encoding="utf-8")

In [25]:

vocab={'<unk>': 0, '<pad>': 1, '，': 2, '的': 3, '。': 4, '是':5}
embedding_vec = [line.replace("\n", "") for line in lines if line.split(" ")[0] in vocab]

In [28]:
len(embedding_vec)
#embedding_vec
#embedding_vec

4

In [21]:

lines = codecs.open(path, encoding="utf-8")
embedding_vec = [line for line in lines ]
print(len(embedding_vec))
#embedding_vec

21131


In [23]:
for one in embedding_vec:
    if one.split(" ")[0] in vocab:
        print(one.split(" ")[0])

。
是
的
，


In [6]:
#lines = codecs.open(path, encoding="utf-8")
#for line in list(lines)[0:10]:
#    print(line[0])

In [17]:
vocab["。"]

4

In [19]:
if "。" in vocab:
    print("yes")

yes
